## <b>Aprendizado de Máquina
<i>Modelo de classificação</b><br>
para</i> Risco de crédito<br><br>
Importação da biblioteca pandas e pré-visualização dos dados:

In [2]:
import pandas as pd

df= pd.read_csv("https://raw.githubusercontent.com/redsonlopez/ds/main/datasets/german_credit.csv")
print(f"O conjunto de dados possui {df.shape[0]} amostras com {df.shape[1]} atributos em cada uma delas: \n")
df.head()

O conjunto de dados possui 1000 amostras com 11 atributos em cada uma delas: 



,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,2,49,male,1,own,little,NaN,2096,12,education,good
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,4,53,male,2,free,little,little,4870,24,car,bad


Criando um valor binario para nosso alvo:

In [3]:
import numpy as np
df['Target']= np.where(df['Risk'] == 'bad', 1, 0)

Separação do conjunto de dados e de validação:

In [4]:
from sklearn import model_selection

features = ["Age",	"Sex",	"Job",	"Housing",	"Saving accounts",	"Checking account",	"Credit amount",	"Duration",	"Purpose"	]
X = pd.get_dummies(df[features])
y = df["Target"]

X_train, X_validation, y_train, y_validation = model_selection.train_test_split(X, y,  test_size= 0.20, random_state= 7)

Isso dividirá nosso conjunto de dados em 10 partes, treinará em 9 e testará em 1 e repetirá para todas as combinações de divisões de teste de treinamento:

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Algoritmos de verificação pontual
models= []
models.append(('RFC', RandomForestClassifier()))
models.append(('LR', LogisticRegression(solver= 'liblinear', multi_class= 'ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma= 'auto')))

# Avalia cada modelo por vez
results= []
names= []
for name, model in models:
   kfold= model_selection.KFold(n_splits= 10, random_state= 7, shuffle= True)
   cv_results= model_selection.cross_val_score(model, X_train, y_train, cv= kfold, scoring= 'accuracy')
   results.append(cv_results)
   names.append(name)
   msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
   print(msg)

RFC: 0.711250 (0.047582)
LR: 0.717500 (0.053092)
LDA: 0.718750 (0.045843)
KNN: 0.603750 (0.055071)
CART: 0.657500 (0.047500)
NB: 0.666250 (0.044388)
SVM: 0.688750 (0.052574)


Calculando a ácuracia do modelo:

In [6]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Previsões no conjunto de dados de validação
Lda= LinearDiscriminantAnalysis()
Lda.fit(X_train, y_train)
predictions = Lda.predict(X_validation)
print("Accuracy:", accuracy_score(y_validation, predictions), "% \n")
print("Confusion Matrix: \n", confusion_matrix(y_validation, predictions), "\n")
print(classification_report(y_validation, predictions))

Accuracy: 0.77 % 

Confusion Matrix: 
 [[129  20]
 [ 26  25]] 

              precision    recall  f1-score   support

           0       0.83      0.87      0.85       149
           1       0.56      0.49      0.52        51

    accuracy                           0.77       200
   macro avg       0.69      0.68      0.68       200
weighted avg       0.76      0.77      0.77       200



Podemos ver a porcentagem da acurácia; a matriz de confusão fornece uma indicação dos erros cometidos; e por fim, o relatório de classificação fornece uma divisão de cada classe por precisão, recall, pontuação f1 e suporte, mostrando seus resultados.

In [7]:
tn, fp, fn, tp= confusion_matrix(y_validation, predictions).ravel()
print("True Positive: " + str(tp))
print("True Negative: " + str(tn))
print("False Positive: " + str(fp))
print("False Negative: " + str(fn))

True Positive: 25
True Negative: 129
False Positive: 20
False Negative: 26


Conjunto processado pelo modelo:

In [8]:
output = pd.DataFrame({'Validations': y_validation, 'Predictions': predictions})
print(output.head())

     Validations  Predictions
778            0            0
334            1            1
271            0            0
802            0            1
216            0            0
